In [207]:
import Localization
from Localization import KalmanFilter
from Localization import NoiseModel
from Localization import MotionModel

In [208]:
import numpy as np
from numpy.linalg import norm as norm
from numpy.linalg import det as matrix_det
from numpy.linalg import inv as matrix_inv
from numpy import arctan2 as atan2

In [209]:
filt = KalmanFilter(0.05, [0,0,0])

sensor covariance matrix :  [1, 1, 1]


In [247]:
map_features = np.arange(12).reshape((4,3)) + .1
z_i = np.arange(2,17).reshape((5,3)) /3
last_state_mu = [[2.2], [5.3], [0.7]]
k = np.shape(map_features)[0]
jacobian_H = np.zeros((4, 3, 3))
innovation_S = np.zeros((k, 3, 3))
sensor_covariance = np.diag([1, 1, 1])
last_covariance = np.arange(1,10).reshape((3,3)) /3
map_features
last_covariance
z_i
np.shape(z_i)[0]

5

In [357]:
delta_x = (map_features[:, 0] - last_state_mu[0])
delta_y = (map_features[:, 1] - last_state_mu[1])
np.array([delta_x, delta_y])
norm(np.array([delta_x, delta_y]), axis=0)
z_hat[:, 0] = norm(np.array([delta_x, delta_y]), axis=0)
# z_hat = np.power(delta_x ** 2 delta_y ** 2)
# z_hat[:, 0] = norm(map_features[:, :2], axis=1)
# delta_x = (map_features[:, 0] - last_state_mu[0])
# delta_y = (map_features[:, 1] - last_state_mu[1])
# z_hat[:, 0].reshape(k, 1)
# delta_x

In [358]:
z_hat[:, 0]

array([4.69574275, 1.5       , 4.29534632, 8.40535544])

In [250]:
jacobian_H[:, 0, 0] = -1 * np.divide(delta_x, z_hat[:, 0])
jacobian_H[:, 0, 1] = -1 * np.divide(delta_y, z_hat[:, 0])
jacobian_H
-1 * np.divide(delta_x, z_hat[:, 0])

array([ 1.90125067, -0.17509595, -0.41664194, -0.50754506])

In [251]:
jacobian_H

array([[[ 1.90125067,  3.80250133,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[-0.17509595,  0.23346127,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[-0.41664194, -0.19229628,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[-0.50754506, -0.35307483,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]])

In [252]:
z_hat[:, 1] = atan2(delta_y, delta_x) - last_state_mu[2]
z_hat[:, 1]
# atan2(delta_y, delta_x)- last_state_mu[2]

array([-2.73444394, -1.62729522, -0.26759222, -0.092198  ])

In [253]:
jacobian_H[:, 1, 0] = np.divide(delta_y, np.power(delta_x, 2) + np.power(delta_y, 2))
jacobian_H[:, 1, 1] = -1 * jacobian_H[:, 1, 0]
jacobian_H[:, 1, 2] = -1

In [353]:
last_state_mu
z_hat

array([[ 1.1045361 , -2.73444394,  2.1       ],
       [ 5.14003891, -1.62729522,  5.1       ],
       [ 9.36055554, -0.26759222,  8.1       ],
       [13.59485197, -0.092198  , 11.1       ]])

In [255]:
innovation_S = jacobian_H @ last_covariance @ np.transpose(jacobian_H, axes=[0, 2, 1]) + sensor_covariance
innovation_S

array([[[ 40.76229508,  -9.14411035,   0.        ],
        [-13.48982616,   3.74603175,   0.        ],
        [  0.        ,   0.        ,   1.        ]],

       [[  1.01930356,  -0.28145053,   0.        ],
        [ -0.18287799,   3.28888889,   0.        ],
        [  0.        ,   0.        ,   1.        ]],

       [[  1.27973066,   0.82103736,   0.        ],
        [  1.54436321,   4.1300813 ,   0.        ],
        [  0.        ,   0.        ,   1.        ]],

       [[  1.65203982,   1.23318505,   0.        ],
        [  2.18427568,   4.0905874 ,   0.        ],
        [  0.        ,   0.        ,   1.        ]]])

In [256]:
likelihood_scores = np.power(2 * np.pi * matrix_det(innovation_S), -0.5)
likelihood_scores
matrix_det(innovation_S)

array([29.3443924 ,  3.30090504,  4.01741175,  4.06419719])

In [310]:
innovation_S_inv = matrix_inv(innovation_S)
innovation_S_inv
# z_i[0] - z_hat[0]
# innovation_S_inv[0, :, :]
# (z_i[0] - z_hat[0])
# # (z_i[0] - z_hat[0]) @ innovation_S_inv[0, :, :]
# # (z_i[0] - z_hat) @ innovation_S_inv
# # np.tensordot((z_i[0] - z_hat),innovation_S_inv, axes=([1,2],[1,2]))
# np.tensordot((z_i[0] - z_hat[0]), innovation_S_inv, axes=([0, 1], [0, 1]))

array([[[ 0.1276575 ,  0.31161355,  0.        ],
        [ 0.45970712,  1.38909999,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[ 0.99635974,  0.08526465,  0.        ],
        [ 0.05540238,  0.30879518,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[ 1.02804531, -0.20436973,  0.        ],
        [-0.38441746,  0.31854605, -0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[ 1.00649334, -0.30342648,  0.        ],
        [-0.53744333,  0.40648614, -0.        ],
        [ 0.        ,  0.        ,  1.        ]]])

In [348]:
number_obs = np.shape(z_i)[0]
number_pred = np.shape(z_hat)[0]
likelihood_scores = np.zeros(number_obs)
number_pred
z_hat[0, :]
z_i[0]

# for each observed feature a likelihood score is computed w.r.t. each predicted feature

for observation in range(number_obs):
    scores = np.zeros(number_pred)
    
    for prediction in range(number_pred):
        delta_z = z_i[observation, :] - z_hat[prediction, :]
        scores[prediction] = scaling_factor[prediction] * np.exp(-0.5 * delta_z @ innovation_S_inv[prediction, :, :] @ delta_z.T)
    print(scores)
    likelihood_scores[observation] = np.argmax(scores)
print(likelihood_scores)

[6.32988462e-06 6.72005709e-09 9.18383317e-30 2.34686119e-61]
[4.36094222e-09 3.73120063e-06 1.89149911e-24 3.69692655e-54]
[1.12142613e-13 1.79521829e-04 6.71712834e-20 1.20902613e-47]
[1.07638295e-19 7.48477844e-04 4.11297906e-16 8.20865486e-42]
[3.85628166e-27 2.70416791e-04 4.34234873e-13 1.15704367e-36]
[0. 1. 1. 1. 1.]


In [350]:
np.eye(3) 

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [ ]:
### compute the likelihood score ###
# pre-compute scaling factor of formula and inverted innovation matrix upfront
scaling_factor = np.power(2 * np.pi * matrix_det(innovation_S), -0.5)
innovation_S_inv = matrix_inv(innovation_S)

# for each pair of observed feature and predicted feature a likelihood score is computed
for observation in range(np.shape(z_i)[0]):
    print(z_i[observation])
    delta_z = z_i[observation] - z_hat
#     print(delta_z)
    max_score = scaling_factor * np.exp(-0.5 * delta_z @ innovation_S_inv @ delta_z.T)
# likelihood_scores[observation]


[0.66666667 1.         1.33333333]
ERROR! Session/line number was not unique in database. History logging moved to new session 39


In [260]:
scaling_factor

array([0.07364572, 0.21958039, 0.19903841, 0.19788947])

In [237]:
a = np.array([[[2, 2], [1, 1]], [[2, 3], [1, 1]]])
b= matrix_det(a)
b

array([ 0., -1.])

In [242]:
innovation_S_inv = np.zeros_like(a)
for prediction in range(np.shape(a)[0]):
    try:
        innovation_S_inv[prediction, :, :] = matrix_inv(a[prediction, :, :])
    except np.linalg.LinAlgError:
        innovation_S_inv[prediction, :, :] = np.eye(np.shape(a)[1])
        
#         innovation_S_inv[prediction, :, :] = np.ones_like(innovation_S_inv[prediction, :, :])
innovation_S_inv

array([[[ 1,  0],
        [ 0,  1]],

       [[-1,  3],
        [ 1, -2]]])

In [245]:
number_obs = np.shape(z_i)[0]
likelihood_scores = np.zeros(number_obs)
likelihood_scores

array([0., 0., 0., 0., 0.])

In [ ]:
    def correctionStep(self, map_features, z_i):
        """
        This method predicts the features measured by the range finder given a pose estimate
        @param:
            *map_features: numpy array of dim (k, 3) containing a subset of k features from the map, which are good
            candidates that may be observed given current robot pose, and where axis 1 contains in order m_x, m_y, m_s

            *z_i: numpy array of dim (i, 3) containing i features extracted from the laser readings,
            where the axis 1 contains in order r, phi, s

        @result: the method returns:
            *z_hat: numpy array of dim (k, 3) containing the predicted measurements,
            where the axis 1 contains in order r, phi, s

            *jacobian_H: numpy array of dim (k, 3, 3) containing the jacobian of the predicted measurements

            *innovation_S: numpy array of dim (k, 3, 3) containing the innovation matrix of the predicted measurements

            *self.last_covariance: covariance of state variables corrected by the measurements

            *self.last_state_mu: state estimate corrected by the measurements
        """
        ### initialize matrices and indices ###

        # number of predictions to be computed
        number_pred = np.shape(map_features)[0]

        # number of observations made
        number_obs = np.shape(z_i)[0]

        # z_hat: numpy array of dim (k, 3) containing the predicted measurements
        z_hat = np.zeros_like(number_pred)

        # jacobian_H: numpy array of dim (k, 3, 3) containing the jacobian of the predicted measurements
        jacobian_H = np.zeros((number_pred, 3, 3))

        # innovation_S: numpy array of dim (k, 3, 3) containing the innovation matrix of the predicted measurements
        innovation_S = np.zeros((number_pred, 3, 3))

        ### compute predicted measurements and corresponding jacobian ###

        # compute r of each predicted measurements
        # norm-2 of the vector from robot's pose to landmark
        delta_x = (map_features[:, 0] - self.last_state_mu[0])
        delta_y = (map_features[:, 1] - self.last_state_mu[1])
        z_hat[:, 0] = norm(np.array([delta_x, delta_y]), axis=0)

        # compute partial derivatives of r
        # dr/du_x = 0.5 * (1/r) * -2 * (m_x - u_x)
        # dr/du_y = 0.5 * (1/r) * -2 * (m_y - u_y)
        jacobian_H[:, 0, 0] = -1 * np.divide(delta_x, z_hat[:, 0])
        jacobian_H[:, 0, 1] = -1 * np.divide(delta_y, z_hat[:, 0])

        # compute phi
        z_hat[:, 1] = atan2(delta_y, delta_x) - self.last_state_mu[2]

        # compute partial derivatives of phi as per the chain rule
        # and the formula of the partial derivatives given on wikipedia: https://en.wikipedia.org/wiki/Atan2
        # dphi / du_x = datan2 / ddelta_x * ddelta_x / du_x = -1 * delta_y / (delta_x^2 + delta_y^2) * -1 = delta_y / (delta_x^2 + delta_y^2)
        # dphi / du_y = datan2 / ddelta_y * ddelta_y / du_y = delta_y / (delta_x^2 + delta_y^2) * -1 = -1 * dphi / du_x
        # dphi / du_psi = -1
        jacobian_H[:, 1, 0] = np.divide(delta_y, np.power(delta_x, 2) + np.power(delta_y, 2))
        jacobian_H[:, 1, 1] = -1 * jacobian_H[:, 1, 0]
        jacobian_H[:, 1, 2] = -1

        # compute s
        z_hat[:, 2] = map_features[:, 2]

        ### compute innovation matrices and initialize its inverse ###

        jacobian_H_transposed = np.transpose(jacobian_H, axes=[0, 2, 1])
        innovation_S = jacobian_H @ self.last_covariance @ jacobian_H_transposed + self.sensor_covariance
        innovation_S_inv = np.zeros_like(innovation_S)

        ### compute the likelihood score ###

        # pre-compute scaling factor of formula upfront
        determinant = matrix_det(innovation_S)
        # if determinant = 0 we set to 1 to avoid division by zero
        determinant[determinant == 0] = 1
        scaling_factor = np.power(2 * np.pi * matrix_det(determinant), -0.5)

        # pre-compute inverted innovation matrix upfront
        # catch error if matrix can't be inverted
        for prediction in range(np.shape(z_hat)[0]):
            try:
                innovation_S_inv[prediction, :, :] = matrix_inv(innovation_S[prediction, :, :])
            except numpy.linalg.LinAlgError:
                innovation_S_inv[prediction, :, :] = np.eye(np.shape(innovation_S_inv)[1])

        # for each observed feature a likelihood score is computed w.r.t. each predicted feature
        # t
        for observation_idx in range(number_obs):
            scores = np.zeros_like(number_pred)
            observation = z_i[observation_idx, :]

            for prediction_idx in range(number_pred):
                prediction = z_hat[prediction_idx, :]
                delta_z = observation - prediction

                # a set of likelihood scores is computed for each observation
                scores[prediction_idx] = scaling_factor[prediction_idx] * \
                                     np.exp(-0.5 * delta_z @ innovation_S_inv[prediction_idx, :, :] @ delta_z.T)

            # for each observed feature the index of the most likely among k features is retained
            most_likely_feature = np.argmax(scores)

            # compute Kalman gain for this observation
            kalman_gain = self.last_covariance @ jacobian_H[most_likely_feature, :, :].T @ innovation_S_inv[most_likely_feature, :, :]

            # correct pose and covariance with respect to this observation
            self.last_state_mu += kalman_gain * (observation - z_hat[most_likely_feature, :])
            self.last_covariance = (np.eye(3) - kalman_gain * jacobian_H[most_likely_feature, :, :]) @ self.last_covariance

        return